#imports

In [ ]:
import torch
import torch.nn
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from collections import Counter

In [ ]:
!wget https://huggingface.co/datasets/iix/Parquet_FIles/resolve/main/CL_de-en.parquet -O CL_de-en.parquet

--2023-10-24 16:04:39--  https://huggingface.co/datasets/iix/Parquet_FIles/resolve/main/CL_de-en.parquet
Resolving huggingface.co (huggingface.co)... 18.172.134.124, 18.172.134.4, 18.172.134.88, ...
Connecting to huggingface.co (huggingface.co)|18.172.134.124|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/fc/9a/fc9a918f6fbc19623c4d012eb54560ce331bef7d7208a8ec162d38c5b4a37971/fef73cfc728100a32c8d71bd9ff2b9c59326db9405d500ee29af8c854c0fb8b1?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27CL_de-en.parquet%3B+filename%3D%22CL_de-en.parquet%22%3B&Expires=1698422679&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTY5ODQyMjY3OX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy9mYy85YS9mYzlhOTE4ZjZmYmMxOTYyM2M0ZDAxMmViNTQ1NjBjZTMzMWJlZjdkNzIwOGE4ZWMxNjJkMzhjNWI0YTM3OTcxL2ZlZjczY2ZjNzI4MTAwYTMyYzhkNzFiZDlmZjJiOWM1OTMyNmRiOTQwNWQ1MDBlZTI5YWY4Yzg1NGMwZmI

In [ ]:
df = pd.read_parquet('CL_de-en.parquet')

In [ ]:
df.sample(5)

,de,en
94252,"Sobald die Verträge unterzeichnet sind, erfolg...","Once the contracts are signed, projects usuall..."
50978,Diesen Software-Technologie-Projekten sollen i...,ECU 35.96 million of the financing will go to ...
24645,Der Beratende Ausschuß für industrielle Forsch...,The European Commission's Industrial Research ...
7383,Außerdem wurde im Zusammenhang mit dem RP7 ein...,A working paper has also been prepared in the ...
92905,Sie sind in einem erweiterten Europa im Hinbli...,They are of increasing importance in an enlarg...


#hyper-params

In [ ]:
dl_batch_size = 32  # dataloader batch size

In [ ]:
#vocab_size = 10000  # Adjust based on your dataset

In [ ]:
embedding_dim = 200  # Adjust as needed

In [ ]:
context_size = 2  # Adjust the context window size

In [ ]:
num_samples = 5  # Number of negative samples per target-context pair

In [ ]:
weights = torch.Tensor([0.75, 0.25])  # Weights for positive and negative samples, can be adjusted

In [ ]:
#word_frequencies = torch.Tensor([0.1, 0.5, 0.01, ...])  # Adjust based on your dataset

In [ ]:
subsampling_threshold = 1e-5

In [ ]:
lr = 0.01

In [ ]:
num_epochs = 10  # Adjust as needed

#code#

In [ ]:
# Assuming 'df' contains columns 'source' and 'target' for source and target sentences.

class TranslationDataset(Dataset):
    def __init__(self, df):
        self.df = df

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        source_sentence = self.df.iloc[idx]['source']
        target_sentence = self.df.iloc[idx]['target']
        return source_sentence, target_sentence

# Create an instance of the TranslationDataset
translation_dataset = TranslationDataset(df)

# Create a DataLoader for batching and shuffling
data_loader = DataLoader(translation_dataset, batch_size=dl_batch_size, shuffle=True)

# Now, 'data_loader' can be used for training your Word2Vec model.


In [ ]:
# Assuming 'df' contains columns 'de' and 'en' for the respective text
german_text = df['de'].str.cat()  # Combine all German sentences
english_text = df['en'].str.cat()  # Combine all English sentences

# Tokenize words in both languages
german_words = german_text.split()  # Tokenize German words
english_words = english_text.split()  # Tokenize English words

# Calculate word frequencies using a Counter for both languages
german_frequencies = Counter(german_words)
english_frequencies = Counter(english_words)

# Combine the counts for both languages
combined_frequencies = german_frequencies + english_frequencies

# Convert to a tensor
word_frequencies = torch.Tensor([combined_frequencies[word] for word in vocab])

# Assuming 'german_words' and 'english_words' have been defined previously
# Combine the words from both languages
all_words = german_words + english_words

# Create a vocabulary of unique words
vocabulary = list(set(all_words))

vocab_size = len(vocabulary)

In [ ]:
class Word2Vec(nn.Module):
    def __init__(self, vocab_size, embedding_dim, context_size, cbow=True):
        super(Word2Vec, self).__init()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.cbow = cbow

    def forward(self, target, context):
        if self.cbow:
            # CBOW model
            target_embedding = self.embedding(context)
            context_out = self.embedding(target)
            scores = torch.mm(target_embedding, context_out.t())
        else:
            # Skip-gram model
            target_embedding = self.embedding(target)
            context_out = self.embedding(context)
            scores = torch.mm(target_embedding, context_out.t())

        return scores

def negative_sampling_loss(input, target, num_samples, weights):
    loss = F.cross_entropy(input, target, weight=weights, reduction='mean')
    return loss

In [ ]:
loss_function = negative_sampling_loss

In [ ]:
skip_gram_model = Word2Vec(vocab_size, embedding_dim, context_size, cbow=False)

In [ ]:
# Define the subsampling threshold
subsampling_threshold = 1e-5

# Calculate word_probs
total_word_count = sum(word_frequencies)
word_probs = torch.sqrt(subsampling_threshold / (word_frequencies / total_word_count))

In [ ]:
# Define an optimizer (e.g., SGD) and move the model to the device (e.g., GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
skip_gram_model.to(device)
optimizer = optim.SGD(skip_gram_model.parameters(), lr = lr)

for epoch in range(num_epochs):
    for target, context in data_loader:
        # Move data to the same device as the model
        target, context = target.to(device), context.to(device)

        optimizer.zero_grad()
        scores = skip_gram_model(target, context)

        # Implement your loss function here
        loss = loss_function(scores, context, num_samples, weights)

        loss.backward()
        optimizer.step()

In [ ]:
cbow_model = Word2Vec(vocab_size, embedding_dim, context_size, cbow=True)
cbow_model.to(device)  # Move the model to the appropriate device

# Define an optimizer for CBOW
cbow_optimizer = optim.SGD(cbow_model.parameters(), lr = lr)

# Training loop for CBOW
for epoch in range(num_epochs):
    for target, context in data_loader:
        target, context = target.to(device), context.to(device)
        cbow_optimizer.zero_grad()
        scores = cbow_model(target, context)
        loss = loss_function(scores, context, num_samples, weights)
        loss.backward()
        cbow_optimizer.step()